In [14]:
from keras import layers
from keras import models
from keras.datasets import cifar10
import matplotlib.pylab as plt
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
import tensorflow as tf
from keras.applications import vgg16
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [2]:
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()
train_data_2, val_data, train_labels_2, val_labels = train_test_split(train_data, train_labels, test_size=0.2)

In [3]:
vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape= (32, 32, 3))

for layer in vgg_conv.layers:
    layer.trainable = False

vgg_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [4]:
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000028DD682F8E0> False
<keras.layers.convolutional.Conv2D object at 0x0000028DFE9AFA30> False
<keras.layers.convolutional.Conv2D object at 0x0000028D89344040> False
<keras.layers.pooling.MaxPooling2D object at 0x0000028D893453C0> False
<keras.layers.convolutional.Conv2D object at 0x0000028D89344640> False
<keras.layers.convolutional.Conv2D object at 0x0000028D89346E90> False
<keras.layers.pooling.MaxPooling2D object at 0x0000028D89347820> False
<keras.layers.convolutional.Conv2D object at 0x0000028D893457B0> False
<keras.layers.convolutional.Conv2D object at 0x0000028D89347A30> False
<keras.layers.convolutional.Conv2D object at 0x0000028D953689D0> False
<keras.layers.pooling.MaxPooling2D object at 0x0000028D9536AF80> False
<keras.layers.convolutional.Conv2D object at 0x0000028D9536AB00> False
<keras.layers.convolutional.Conv2D object at 0x0000028D9536A4A0> False
<keras.layers.convolutional.Conv2D object at 0x0000028D9537CC40> False
<ker

In [5]:
train_labels

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [6]:
model = Sequential()
model.add(vgg_conv)
model.add(Flatten(input_shape=(None, 64, 64, 3)))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 14,719,818
Trainable params: 5,130
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
batch_size = 32

In [28]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history = model.fit(train_data_2, train_labels_2, epochs=20, batch_size=batch_size)

Epoch 1/20
1250/1250 [==============================] - 100s 78ms/step - loss: 1.9205 - accuracy: 0.5330
Epoch 2/20
1250/1250 [==============================] - 93s 75ms/step - loss: 1.8700 - accuracy: 0.5391
Epoch 3/20
1250/1250 [==============================] - 95s 76ms/step - loss: 1.8952 - accuracy: 0.5310
Epoch 4/20
1250/1250 [==============================] - 100s 80ms/step - loss: 1.8778 - accuracy: 0.5321
Epoch 5/20
1250/1250 [==============================] - 105s 84ms/step - loss: 1.8790 - accuracy: 0.5374
Epoch 6/20
1250/1250 [==============================] - 98s 79ms/step - loss: 1.8935 - accuracy: 0.5344
Epoch 7/20
1250/1250 [==============================] - 105s 84ms/step - loss: 1.8732 - accuracy: 0.5350
Epoch 8/20
1250/1250 [==============================] - 99s 80ms/step - loss: 1.9009 - accuracy: 0.5350
Epoch 9/20
1250/1250 [==============================] - 98s 78ms/step - loss: 1.8833 - accuracy: 0.5360
Epoch 10/20
1250/1250 [==============================] - 99s

In [9]:
model_3 = Sequential()
model_3.add(vgg_conv)
model_3.add(Flatten(input_shape=(None, 64, 64, 3)))
model_3.add(Dense(512, activation="relu"))
model_3.add(Dropout(0.25))
model_3.add(Dense(10, activation='softmax'))
model_3.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history = model_3.fit(val_data, val_labels, epochs=10)

Epoch 1/10
313/313 [==============================] - 26s 82ms/step - loss: 5.8392 - accuracy: 0.3931
Epoch 2/10
313/313 [==============================] - 26s 82ms/step - loss: 1.4671 - accuracy: 0.5463
Epoch 3/10
313/313 [==============================] - 26s 83ms/step - loss: 1.0704 - accuracy: 0.6427
Epoch 4/10
313/313 [==============================] - 26s 83ms/step - loss: 0.8605 - accuracy: 0.7008
Epoch 5/10
313/313 [==============================] - 26s 82ms/step - loss: 0.7588 - accuracy: 0.7372
Epoch 6/10
313/313 [==============================] - 26s 82ms/step - loss: 0.6510 - accuracy: 0.7710
Epoch 7/10
313/313 [==============================] - 26s 83ms/step - loss: 0.6024 - accuracy: 0.7838
Epoch 8/10
313/313 [==============================] - 26s 84ms/step - loss: 0.5538 - accuracy: 0.8036
Epoch 9/10
313/313 [==============================] - 26s 84ms/step - loss: 0.5293 - accuracy: 0.8108
Epoch 10/10
313/313 [==============================] - 26s 83ms/step - loss: 0.510

In [11]:
model_5 = Sequential()
model_5.add(vgg_conv)
model_5.add(Flatten(input_shape=(None, 64, 64, 3)))
model_5.add(Dense(512, activation="relu"))
model_5.add(Dropout(0.25))
model_5.add(Dense(10, activation='softmax'))
model_5.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history = model_5.fit(val_data, val_labels, epochs=60)

Epoch 1/60
313/313 [==============================] - 29s 91ms/step - loss: 5.6022 - accuracy: 0.4096
Epoch 2/60
313/313 [==============================] - 26s 84ms/step - loss: 1.4980 - accuracy: 0.5402
Epoch 3/60
313/313 [==============================] - 22s 70ms/step - loss: 1.0887 - accuracy: 0.6342
Epoch 4/60
313/313 [==============================] - 22s 69ms/step - loss: 0.8844 - accuracy: 0.6993
Epoch 5/60
313/313 [==============================] - 22s 71ms/step - loss: 0.7589 - accuracy: 0.7321
Epoch 6/60
313/313 [==============================] - 24s 76ms/step - loss: 0.6681 - accuracy: 0.7687
Epoch 7/60
313/313 [==============================] - 24s 77ms/step - loss: 0.6233 - accuracy: 0.7847
Epoch 8/60
313/313 [==============================] - 22s 71ms/step - loss: 0.5806 - accuracy: 0.7977
Epoch 9/60
313/313 [==============================] - 22s 71ms/step - loss: 0.5431 - accuracy: 0.8112
Epoch 10/60
313/313 [==============================] - 22s 71ms/step - loss: 0.505

In [12]:
test_loss, test_acc = model_5.evaluate(test_data, test_labels)
print('test_acc:', test_acc)

313/313 [==============================] - 28s 88ms/step - loss: 5.6045 - accuracy: 0.5600
test_acc: 0.5600000023841858


In [13]:
test_loss, test_acc = model_3.evaluate(test_data, test_labels)
print('test_acc:', test_acc)

313/313 [==============================] - 25s 79ms/step - loss: 2.0412 - accuracy: 0.5439
test_acc: 0.5439000129699707


In [21]:
data_augmentation = tf.keras.Sequential([
layers.RandomFlip("horizontal", input_shape=(32, 32,3)),
tf.keras.layers.RandomRotation(0.1,fill_mode='reflect',interpolation='bilinear',seed=None,fill_value=0.0),
tf.keras.layers.RandomZoom(0.1,fill_mode='reflect',interpolation='bilinear',seed=None,fill_value=0.0),
])

In [23]:
model_6 = Sequential([
  data_augmentation,
  vgg_conv,
  layers.Flatten(input_shape=(None, 64, 64, 3)),
  layers.Dropout(0.25),
  layers.Dense(10, activation='softmax')
])

model_6.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history_1 = model_6.fit(val_data, val_labels, epochs=100) 

Epoch 1/100
313/313 [==============================] - 29s 89ms/step - loss: 16.6153 - accuracy: 0.2512
Epoch 2/100
313/313 [==============================] - 27s 87ms/step - loss: 7.6858 - accuracy: 0.3545
Epoch 3/100
313/313 [==============================] - 28s 89ms/step - loss: 4.8214 - accuracy: 0.3913
Epoch 4/100
313/313 [==============================] - 30s 96ms/step - loss: 3.5053 - accuracy: 0.3974
Epoch 5/100
313/313 [==============================] - 30s 95ms/step - loss: 2.8081 - accuracy: 0.4105
Epoch 6/100
313/313 [==============================] - 30s 95ms/step - loss: 2.6085 - accuracy: 0.4070
Epoch 7/100
313/313 [==============================] - 31s 98ms/step - loss: 2.4847 - accuracy: 0.4004
Epoch 8/100
313/313 [==============================] - 30s 97ms/step - loss: 2.4424 - accuracy: 0.4046
Epoch 9/100
313/313 [==============================] - 30s 97ms/step - loss: 2.4254 - accuracy: 0.3988
Epoch 10/100
313/313 [==============================] - 30s 96ms/step - 

In [27]:
def myprint(s):
    with open('modelsummary.txt','a') as f:
        print(s, file=f)
model.summary(print_fn=myprint)
model_5.summary(print_fn=myprint)

In [ ]:
model.evaluate(test_data, test_labels)